# 1. Introduction

The purpose of this analysis is to estimate locations for district events in FUM.

There are a number of assumptions going into this analysis:
1. District events are only hosted at high schools with teams.  This is done to reduce the number of distance checks done against the google maps API.
2. The district championship is hosted at Williams Arena with 60 teams.
3. The world championship (and associated travel distances) is outside the scope of this analysis.
4. Teams are only the host for 1 district event.  This simplifies and encourages the search to spread events out.
5. Teams would only be going to two district events.  Third district events are outside the scope of this analysis, but there will be extra slots.
6. This analysis uses a combined dataset from 2023 and 2024 registration for a team count of 202, requiring 11 district events (and therefore 3 fields).  This analysis uses Minnesota, North Dakota, and South Dakota teams as recommended by the FUM PDO committee.
7. This analysis uses 50 miles as a hard cutoff between a "nearby" and a "travel" event for teams, and a flat cost estimate of $6,000 per travel event per team.  The details of team travel are far too complex for this to fully simulate, but this was chosen as an approximation.
8. The 2024 data on the graphs is not yet representative of a full season because many of our teams (including my own) are still on waitlists.  I expect the total travel to be even higher than 2023.

The goal of this analysis is to give a good estimate of locations and travel costs for teams if FUM switched to a district system.  It uses a hard cap of 50 miles as a "nearby" vs a "travel" event.  To estimate the cost to teams, the search aims to give as many teams as possible a "home" event which is within 50 miles, then assigns teams travel events to fill the remaining slots.


# 2. Code

## 2.1. Imports

In [1]:
import json
from frcpy import FRCPy
from frcpy.models import Location, PreciseLocation, Event, Team
import os
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick


## 2.2. Constants

In [2]:
DISTRICT_TEAMS_PER_EVENT = 40
DISTRICT_EVENTS_PER_TEAM = 2
DISTRICT_TEAMS_PER_DISTRICT_CHAMPIONSHIP = 60


In [3]:
MILES_TO_METERS = 1609.344
TRAVEL_DISTANCE = 50.0 * MILES_TO_METERS
TRAVEL_COST = 6000.00


### 2.2.1. State abbreviations

In [4]:
US_STATE_ABBREVIATIONS = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC': 'District of Columbia',
    'MP': 'Northern Mariana Islands',
    'PW': 'Palau',
    'PR': 'Puerto Rico',
    'VI': 'Virgin Islands',
    'AA': 'Armed Forces Americas (Except Canada)',
    'AE': 'Armed Forces Other/Canada/Other/Middle East',
    'AP': 'Armed Forces Pacific'
}


## 2.3. API

In [5]:
try:
    # Load the tokens from the environment
    TBA_TOKEN = os.environ['SECRET_TBA_TOKEN']
    GMAPS_TOKEN = os.environ['SECRET_GMAPS_TOKEN']
except KeyError:
    # Or load from JSON file
    with open('token.json', 'r', encoding='UTF+8') as f:
        tokens = json.load(f)
        TBA_TOKEN = tokens['TBA']
        GMAPS_TOKEN = tokens['GMAPS']
        del tokens
    del f

API = FRCPy(TBA_TOKEN, GMAPS_TOKEN)
del TBA_TOKEN, GMAPS_TOKEN


## 2.4. Support functions

In [6]:
def is_fum(location: Location) -> bool:
    if location.country() != 'USA':
        return False

    state_prov = location.state_prov()
    if state_prov in US_STATE_ABBREVIATIONS.keys():  # Harmonize state names
        state_prov = US_STATE_ABBREVIATIONS[state_prov]
    return state_prov == 'Minnesota' or state_prov == 'North Dakota' or state_prov == 'South Dakota'


## 2.5. Read data from APIs

### 2.5.1. Teams

In [7]:
# Prepare team keys
team_keys = API.teams(cache_expiry=1)
print(f"{len(team_keys):,} teams")


8,293 teams


In [8]:
# Filter for only FUM teams
fum_team_keys: list[str] = []
for team_key in team_keys:
    team = API.team(team_key)
    if is_fum(team.location()):
        fum_team_keys.append(team_key)
    del team_key, team
print(f"Found {len(fum_team_keys)} FUM teams")


Found 327 FUM teams


In [9]:
# Prepare precise team locations
fum_team_locations: dict[str, PreciseLocation] = {}
for team_key in fum_team_keys:
    team = API.team(team_key)
    precise_location = API.team_precise_location(team)
    if precise_location is not None:
        fum_team_locations[team_key] = precise_location
    else:
        print(f"Unable to find precise location for {team_key}")
    del team_key, team, precise_location
print(f"Found {len(fum_team_locations.keys())} precise FUM team locations.")


Found 327 precise FUM team locations.


In [10]:
# Filter for only active FUM teams
active_fum_team_keys: list[str] = []
temp: dict[str, list[str]] = {'Both': [], '2023': [], '2024': []}
for team_key in fum_team_keys:
    team_years = API.team_years(team_key)
    if 2023 in team_years and 2024 in team_years:
        temp['Both'].append(team_key)
    elif 2023 in team_years:
        temp['2023'].append(team_key)
    elif 2024 in team_years:
        temp['2024'].append(team_key)
    del team_key, team_years

print(f"Found {len(temp['Both'])} active FUM teams")
print(f"Found {len(temp['2023'])} retiring FUM teams")
print(f"Found {len(temp['2024'])} new FUM teams")
active_fum_team_keys.extend(temp['Both'])
active_fum_team_keys.extend(temp['2023'])
active_fum_team_keys.extend(temp['2024'])
del temp
print(f"Found {len(active_fum_team_keys)} total active FUM teams")


Found 185 active FUM teams
Found 13 retiring FUM teams
Found 4 new FUM teams
Found 202 total active FUM teams


In [11]:
# Prepare team distance matrix
active_fum_team_distances: dict[tuple[str, str], float] = {}
for team_key_1 in active_fum_team_keys:
    for team_key_2 in active_fum_team_keys:
        if team_key_1 == team_key_2:
            del team_key_2
            continue  # Don't check team to itself
        if (team_key_1, team_key_2) in active_fum_team_distances.keys() or (team_key_2, team_key_1) in active_fum_team_distances.keys():
            del team_key_2
            continue  # Don't check distances that already exist
        distance = API.precise_distance(fum_team_locations[team_key_1], fum_team_locations[team_key_2])
        if distance is None:
            print(f"Unable to check {team_key_1} to {team_key_2}")
            del team_key_2, distance
            continue
        active_fum_team_distances[(team_key_1, team_key_2)] = distance
        active_fum_team_distances[(team_key_2, team_key_1)] = distance
        del team_key_2, distance
    del team_key_1
print(f"Found {len(active_fum_team_distances.keys()):,} precise FUM team distances.")


Found 40,602 precise FUM team distances.


### 2.5.2. Events

In [12]:
# Prepare event keys
event_keys_by_year: dict[int, list[str]] = {}
temp = API.year_range()
for year in range(temp[0], temp[1] + 1):
    event_keys_by_year[year] = API.year_events(year)
    print(f"Found {len(event_keys_by_year[year])} in {year}")
    del year
del temp


Found 1 in 1992
Found 1 in 1993
Found 1 in 1994
Found 2 in 1995
Found 2 in 1996
Found 4 in 1997
Found 6 in 1998
Found 8 in 1999
Found 11 in 2000
Found 18 in 2001
Found 22 in 2002
Found 28 in 2003
Found 32 in 2004
Found 37 in 2005
Found 40 in 2006
Found 45 in 2007
Found 54 in 2008
Found 57 in 2009
Found 58 in 2010
Found 67 in 2011
Found 81 in 2012
Found 128 in 2013
Found 165 in 2014
Found 179 in 2015
Found 203 in 2016
Found 255 in 2017
Found 278 in 2018
Found 303 in 2019
Found 196 in 2020
Found 259 in 2021
Found 288 in 2022
Found 304 in 2023
Found 175 in 2024


In [13]:
# Filter for only FUM events
fum_events: dict[str, Event] = {}
for year, event_keys in event_keys_by_year.items():
    for event_key in event_keys:
        event = API.event(event_key)
        if event.event_type() != 0:
            del event_key, event
            continue
        if is_fum(event.location()):
            fum_events[event_key] = event
        del event_key, event
    del year, event_keys
print(f"Found {len(fum_events)} FUM events")


Found 60 FUM events


In [14]:
# Prepare FUM event precise locations
fum_event_locations: dict[str, PreciseLocation] = {}
for event_key, event in fum_events.items():
    fum_event_locations[event_key] = event.precise_location()
    del event_key, event
print(f"Found {len(fum_event_locations.keys())} FUM event locations")


Found 60 FUM event locations


# 3. Baseline data

## 3.1. Calculations

In [99]:
fum_travel_data: dict[int, tuple[int, int, float]] = {}
for year in event_keys_by_year.keys():
    if year == 2021:
        del year
        continue
    nearby = 0
    travel = 0
    total = 0
    for team_key in fum_team_keys:
        team_years = API.team_years(team_key)
        if year not in team_years:
            del team_key, team_years
            continue

        # Team competed this year, check travel costs
        team_location = fum_team_locations[team_key]
        for event_key in API.team_year_events(team_key, year):
            if event_key in fum_events.keys():
                event = fum_events[event_key]
            else:
                event = API.event(event_key)
            if event.event_type() != 0:
                del event_key, event
                continue

            if event_key in fum_event_locations.keys():
                event_location = fum_event_locations[event_key]
            else:
                event_location = API.event(event_key).precise_location()
            try:
                distance = API.precise_distance(team_location, event_location)
            except BaseException as e:
                MANUAL_TRAVEL_EVENTS = [
                    ('frc874', '2006co'),
                    ('frc878', '2006co'),
                    ('frc1349', '2006co'),
                    ('frc877', '2007co')
                ]
                MANUAL_NEARBY_EVENTS = []
                if (team_key, event_key) in MANUAL_TRAVEL_EVENTS:
                    travel += 1
                elif (team_key, event_key) in MANUAL_TRAVEL_EVENTS:
                    nearby += 1
                else:
                    print(
                        f"Failed to get distance from {team_key} to {event_key}: {e}")
                del MANUAL_TRAVEL_EVENTS, MANUAL_NEARBY_EVENTS, event_key, event_location
                continue
            if distance is None:
                print(f"Unable to get distance from {team_key} to {event_key}")
                del event_key, event, event_location, distance
                continue
            total += distance
            if distance < TRAVEL_DISTANCE:
                nearby += 1
            else:
                travel += 1
            del event_key, event, event_location, distance
        del team_key, team_years, team_location
    if travel + nearby > 0:
        fum_travel_data[year] = (nearby, travel, total)
        print(f"{year}: ${travel * TRAVEL_COST:,.2f} for {(travel + nearby):,} slot{'s' if (travel + nearby) > 1 else ''} (${((travel * TRAVEL_COST) / (travel + nearby)):,.2f} per slot, {((travel * 100.0) / (travel + nearby)):,.2f}%, {travel:,} travel, {nearby:,} nearby, {total / MILES_TO_METERS:,.0f} miles, {total / MILES_TO_METERS / (travel + nearby):,.2f} miles per slot)")
    del year, nearby, travel, total


1998: $6,000.00 for 1 slot ($6,000.00 per slot, 100.00%, 1 travel, 0 nearby, 388 miles, 387.84 miles per slot)
1999: $6,000.00 for 1 slot ($6,000.00 per slot, 100.00%, 1 travel, 0 nearby, 1,621 miles, 1,620.89 miles per slot)
2000: $12,000.00 for 2 slots ($6,000.00 per slot, 100.00%, 2 travel, 0 nearby, 2,045 miles, 1,022.42 miles per slot)
2001: $6,000.00 for 1 slot ($6,000.00 per slot, 100.00%, 1 travel, 0 nearby, 418 miles, 418.40 miles per slot)
2002: $120,000.00 for 20 slots ($6,000.00 per slot, 100.00%, 20 travel, 0 nearby, 16,323 miles, 816.14 miles per slot)
2003: $120,000.00 for 20 slots ($6,000.00 per slot, 100.00%, 20 travel, 0 nearby, 18,266 miles, 913.32 miles per slot)
2004: $114,000.00 for 19 slots ($6,000.00 per slot, 100.00%, 19 travel, 0 nearby, 15,005 miles, 789.75 miles per slot)
2005: $60,000.00 for 10 slots ($6,000.00 per slot, 100.00%, 10 travel, 0 nearby, 8,572 miles, 857.21 miles per slot)
2006: $54,000.00 for 9 slots ($6,000.00 per slot, 100.00%, 9 travel, 0 n

# 4. District events

## 4.1. Data structures

In [100]:
@dataclass
class VenueCandidate:
    home_team_key: str
    nearby_team_keys: list[tuple[str, float]]

    def remove_teams(self, team_keys: list[str]) -> None:
        nearby = []
        for team_key, distance in self.nearby_team_keys:
            if team_key in team_keys:
                del team_key, distance
                continue
            nearby.append((team_key, distance))
            del team_key, distance
        self.nearby_team_keys = nearby
        del nearby

    def __str__(self) -> str:
        return f"{self.home_team_key}: {len(self.nearby_team_keys)}"


In [101]:
@dataclass
class DistrictEvent:
    home_team_key: str
    nearby_teams: dict[str, float]
    travel_teams: dict[str, float]

    def team_count(self) -> int:
        return 1 + len(self.nearby_teams.keys()) + len(self.travel_teams.keys())

    def total_travel_distance(self) -> float:
        result = 0
        for _, distance in self.nearby_teams.items():
            result += distance
            del distance
        for _, distance in self.travel_teams.items():
            result += distance
            del distance
        return result


In [102]:
@dataclass
class DistrictTeam:
    team_key: str
    events: list[DistrictEvent]

    def event_stats(self) -> tuple[int, int, int]:
        hosting = 0
        nearby = 0
        travel = 0
        for event in self.events:
            if self.team_key == event.home_team_key:
                hosting += 1
            elif self.team_key in event.nearby_teams.keys():
                nearby += 1
            elif self.team_key in event.travel_teams.keys():
                travel += 1
        return hosting, nearby, travel

    def __str__(self) -> str:
        return f"{self.team_key}: {len(self.events)}"

    def __lt__(self, other: 'DistrictTeam') -> bool:
        if len(self.events) > len(other.events):
            return False
        elif len(self.events) < len(other.events):
            return True

        our_stats = self.event_stats()
        their_stats = other.event_stats()
        if our_stats[0] > their_stats[0]:
            return False
        elif our_stats[0] < their_stats[0]:
            return True
        if our_stats[1] > their_stats[1]:
            return False
        elif our_stats[1] < their_stats[1]:
            return True
        if our_stats[2] > their_stats[2]:
            return False
        elif our_stats[2] < their_stats[2]:
            return True
        return False


## 4.2. Support functions

In [103]:
def create_venue_candidates() -> list[VenueCandidate]:
    venues: list[VenueCandidate] = []
    for home_team_key in active_fum_team_keys:
        nearby = []
        for other_team_key in active_fum_team_keys:
            if home_team_key == other_team_key:
                continue  # Don't check team to itself
            distance = active_fum_team_distances[(home_team_key, other_team_key)]
            if distance < TRAVEL_DISTANCE:
                nearby.append((other_team_key, distance))
            del other_team_key, distance
        nearby.sort(key=lambda x : x[1])
        venues.append(VenueCandidate(home_team_key, nearby))
        del home_team_key, nearby
    return venues


## 4.3. Find home events

In [104]:
assigned_team_keys: dict[str, dict[str, float]] = {}
unassigned_team_keys: list[str] = active_fum_team_keys.copy()

venue_candidates = create_venue_candidates()
while len(assigned_team_keys.keys()) < (len(active_fum_team_keys) * 2 // DISTRICT_TEAMS_PER_EVENT) + 1:
    # Sort venue candidates
    venue_candidates.sort(
        key=lambda x: Team.team_key_to_number(x.home_team_key))
    venue_candidates.reverse()
    venue_candidates.sort(key=lambda x: len(x.nearby_team_keys))
    venue_candidates.reverse()

    # Select the event host
    event_host_key = venue_candidates[0].home_team_key

    # Select the event teams
    event_assigned_teams = {}
    for team_key, distance in venue_candidates[0].nearby_team_keys[:(DISTRICT_TEAMS_PER_EVENT - 1)]:
        event_assigned_teams[team_key] = distance
        del team_key, distance
    event_assigned_team_keys = list(event_assigned_teams.keys())
    assigned_team_keys[event_host_key] = event_assigned_teams
    del event_assigned_teams

    # Prune the venue candidate
    venue_candidates = venue_candidates[1:]

    # Remove event host and teams from unassigned list
    event_assigned_team_keys.insert(0, event_host_key)
    del event_host_key
    for event_assigned_team_key in event_assigned_team_keys:
        unassigned_team_keys.remove(event_assigned_team_key)
        del event_assigned_team_key

    # Remove event host and teams from all future candidates
    result = []
    for venue_candidate in venue_candidates:
        if venue_candidate.home_team_key in event_assigned_team_keys:
            continue
        venue_candidate.remove_teams(event_assigned_team_keys)
        result.append(venue_candidate)
        del venue_candidate
    venue_candidates = result
    del event_assigned_team_keys, result

    print('-' * 120)
    if len(assigned_team_keys.keys()) > 0:
        print(
            f"{len(assigned_team_keys.keys())} Event{'s' if len(assigned_team_keys.keys()) > 1 else ''}:")
        for event_host_key, event_team_list in assigned_team_keys.items():
            print(f"- {event_host_key}: {len(event_team_list)}")
            del event_host_key, event_team_list
    else:
        print('0 Events, ', end='')
    print(f"{len(venue_candidates)} remaining venues")
del venue_candidates


------------------------------------------------------------------------------------------------------------------------
1 Event:
- frc3630: 39
162 remaining venues
------------------------------------------------------------------------------------------------------------------------
2 Events:
- frc3630: 39
- frc2177: 39
122 remaining venues
------------------------------------------------------------------------------------------------------------------------
3 Events:
- frc3630: 39
- frc2177: 39
- frc6175: 22
99 remaining venues
------------------------------------------------------------------------------------------------------------------------
4 Events:
- frc3630: 39
- frc2177: 39
- frc6175: 22
- frc2512: 10
88 remaining venues
------------------------------------------------------------------------------------------------------------------------
5 Events:
- frc3630: 39
- frc2177: 39
- frc6175: 22
- frc2512: 10
- frc3691: 10
77 remaining venues
----------------------------------

In [105]:
events: list[DistrictEvent] = []
for event_host_key, event_assigned_teams in assigned_team_keys.items():
    events.append(DistrictEvent(event_host_key, event_assigned_teams, {}))
    del event_host_key, event_assigned_teams


In [106]:
teams: list[DistrictTeam] = []
for team_key in unassigned_team_keys:
    teams.append(DistrictTeam(team_key, []))
    del team_key
for event in events:
    teams.append(DistrictTeam(event.home_team_key, [event]))
    for team_key, _ in event.nearby_teams.items():
        teams.append(DistrictTeam(team_key, [event]))
        del team_key
    del event


In [107]:
del assigned_team_keys, unassigned_team_keys


## 4.4. Find travel events

In [108]:
while True:
    teams.sort()

    team = teams[0]
    if len(team.events) == DISTRICT_EVENTS_PER_TEAM:
        break # We've found events for all teams!

    nearest_open_event = None
    for event in events:
        if event.team_count() >= DISTRICT_TEAMS_PER_EVENT or event.home_team_key == team.team_key:
            del event
            continue
        distance = active_fum_team_distances[(team.team_key, event.home_team_key)]
        if nearest_open_event is None or distance < nearest_open_event[1]:
            nearest_open_event = (event, distance)
        del event, distance

    if nearest_open_event is None:
        print(f"Unable to place {team.team_key} in an event!")
        del team, nearest_open_event
        break

    team.events.append(nearest_open_event[0])
    if nearest_open_event[1] < TRAVEL_DISTANCE:
        nearest_open_event[0].nearby_teams[team.team_key] = nearest_open_event[1]
    else:
        nearest_open_event[0].travel_teams[team.team_key] = nearest_open_event[1]
    del team, nearest_open_event


## 4.5. Analysis

In [109]:
print(f"{len(events)} events:")
for event in events:
    print(f"- Hosted by {Team.team_key_to_number(event.home_team_key)} in {API.team(event.home_team_key).location().city()} with {event.team_count()} teams ({1 + len(event.nearby_teams.keys())} nearby & {len(event.travel_teams.keys())} travel)")
    del event


11 events:
- Hosted by 3630 in Minneapolis with 40 teams (40 nearby & 0 travel)
- Hosted by 2177 in Mendota Heights with 40 teams (40 nearby & 0 travel)
- Hosted by 6175 in Eden Valley-Watkins with 40 teams (23 nearby & 17 travel)
- Hosted by 2512 in Duluth with 20 teams (11 nearby & 9 travel)
- Hosted by 3691 in Northfield with 40 teams (34 nearby & 6 travel)
- Hosted by 5913 in Pequot Lakes with 24 teams (9 nearby & 15 travel)
- Hosted by 3212 in Granite Falls with 32 teams (9 nearby & 23 travel)
- Hosted by 3278 in Detroit Lakes with 19 teams (8 nearby & 11 travel)
- Hosted by 4181 in Blackduck with 11 teams (7 nearby & 4 travel)
- Hosted by 2847 in Fairmont with 23 teams (6 nearby & 17 travel)
- Hosted by 5464 in Cambridge with 40 teams (21 nearby & 19 travel)


In [110]:
team_travel_categories: dict[str, list[DistrictTeam]] = {
    'Double nearby': [], 'Mixed': [], 'Double travel': []}
total_slots = 0
total_travel = 0
for team in teams:
    hosting, nearby, travel = team.event_stats()
    total_travel += travel
    total_slots += hosting + nearby + travel
    assert hosting + nearby + travel == DISTRICT_EVENTS_PER_TEAM
    match travel:
        case 2:
            team_travel_categories['Double travel'].append(team)
        case 1:
            team_travel_categories['Mixed'].append(team)
        case 0:
            team_travel_categories['Double nearby'].append(team)
    del team, hosting, nearby, travel

for key, team_list in team_travel_categories.items():
    print(f"{key}: {len(team_list)}")
    del key, team_list
print(f"Total cost: ${total_travel * TRAVEL_COST:,.2f} for {total_slots} slots (${total_travel * TRAVEL_COST / total_slots:,.2f} per slot, {total_travel * 100.0 / total_slots:.2f}%, {total_travel} travel, {total_slots - total_travel} nearby)")
district_travel_data = (total_slots, total_travel, total_slots - total_travel)
del team_travel_categories, total_slots, total_travel


Double nearby: 81
Mixed: 86
Double travel: 35
Total cost: $936,000.00 for 404 slots ($2,316.83 per slot, 38.61%, 156 travel, 248 nearby)


# 5. District Championship

In [111]:
team_epas: list[tuple[str, float]] = []
for team_key in active_fum_team_keys:
    try:
        team_stats = API.team_year_stats(team_key, 2024)
    except UserWarning:
        team_stats = API.team_year_stats(team_key, 2023)
    team_epas.append((team_key, team_stats.norm_epa_end()))
    del team_key, team_stats
team_epas.sort(key=lambda x: x[1], reverse=True)


In [112]:
event_location = fum_event_locations['2024mnmi']
travel = 0
nearby = 0
district_championship_teams: dict[str, float] = {}
for team_key, _ in team_epas[:DISTRICT_TEAMS_PER_DISTRICT_CHAMPIONSHIP]:
    distance = API.precise_distance(
        fum_team_locations[team_key], event_location)
    if distance is None:
        print(f"Unable to find distance from {team_key} to {event_location}")
        del team_key, distance
        continue
    district_championship_teams[team_key] = distance
    if distance < TRAVEL_DISTANCE:
        travel += 1
    else:
        nearby += 1
    del team_key, distance
print(f"District championship at {fum_events['2024mnmi'].location_name()} with {travel + nearby} teams, {travel} travel and {nearby} nearby")
print(f"Cost: ${travel * TRAVEL_COST:,.2f} for {travel + nearby} slots (${travel * TRAVEL_COST / (travel + nearby):,.2f} per slot, {travel * 100.0 / (travel + nearby):.2f}%, {travel} travel, {nearby} nearby)")
district_travel_data = (district_travel_data[0] + travel + nearby,
                        district_travel_data[1] + travel, district_travel_data[2] + nearby)
del event_location, travel, nearby


District championship at TDB Builders with 60 teams, 32 travel and 28 nearby
Cost: $192,000.00 for 60 slots ($3,200.00 per slot, 53.33%, 32 travel, 28 nearby)


# 6. Analysis

In [119]:
data = {
    'Year': [],
    'Travel slots': [],
    'Nearby slots': [],
    'Total distance': [],
    'Color': []
}
for year, (nearby, travel, total) in fum_travel_data.items():
    data['Year'].append(year)
    data['Travel slots'].append(travel)
    data['Nearby slots'].append(nearby)
    data['Total distance'].append(total / MILES_TO_METERS)
    data['Color'].append('Blue')
    del year, nearby, travel, total
data['Year'].append('Districts')
data['Travel slots'].append(district_travel_data[1])
data['Nearby slots'].append(district_travel_data[2])
data['Color'].append('Orange')
total = 0
for event in events:
    total += event.total_travel_distance()
    del event
for _, distance in district_championship_teams.items():
    total += distance
    del distance
data['Total distance'].append(total / MILES_TO_METERS)
del total

fum_travel_data_df = pd.DataFrame(data)
del data
fum_travel_data_df['Total slots'] = fum_travel_data_df['Nearby slots'] + \
    fum_travel_data_df['Travel slots']
fum_travel_data_df['Travel percent'] = fum_travel_data_df['Travel slots'] / \
    fum_travel_data_df['Total slots'] * 100
fum_travel_data_df['Travel costs'] = fum_travel_data_df['Travel slots'] * TRAVEL_COST
fum_travel_data_df['Travel cost per slot'] = fum_travel_data_df['Travel costs'] / \
    fum_travel_data_df['Total slots']
fum_travel_data_df['Travel distance per slot'] = fum_travel_data_df['Total distance'] / \
    fum_travel_data_df['Total slots']
fum_travel_data_df.to_csv('fum-regional-vs-district-travel.csv')


In [120]:
data = {
    'Team': [],
    'Event 1 Host': [],
    'Event 1 Type': [],
    'Event 1 Distance': [],
    'Event 2 Host': [],
    'Event 2 Type': [],
    'Event 2 Distance': [],
    'District Championship': [],
    'District Championship Type': [],
    'District Championship Distance': [],
}
for team in teams:
    data['Team'].append(team.team_key)
    event_1 = team.events[0]
    data['Event 1 Host'].append(event_1.home_team_key)
    if event_1.home_team_key == team.team_key:
        data['Event 1 Type'].append('Host')
        data['Event 1 Distance'].append(0.0)
    elif team.team_key in event_1.nearby_teams.keys():
        data['Event 1 Type'].append('Nearby')
        data['Event 1 Distance'].append(event_1.nearby_teams[team.team_key])
    else:
        data['Event 1 Type'].append('Travel')
        data['Event 1 Distance'].append(event_1.travel_teams[team.team_key])
    del event_1
    event_2 = team.events[1]
    data['Event 2 Host'].append(event_2.home_team_key)
    if event_2.home_team_key == team.team_key:
        data['Event 2 Type'].append('Host')
        data['Event 2 Distance'].append(0.0)
    elif team.team_key in event_2.nearby_teams.keys():
        data['Event 2 Type'].append('Nearby')
        data['Event 2 Distance'].append(event_2.nearby_teams[team.team_key])
    else:
        data['Event 2 Type'].append('Travel')
        data['Event 2 Distance'].append(event_2.travel_teams[team.team_key])
    del event_2
    if team.team_key not in district_championship_teams.keys():
        data['District Championship'].append(False)
        data['District Championship Type'].append('')
        data['District Championship Distance'].append(-1)
    else:
        data['District Championship'].append(True)
        data['District Championship Type'].append('Nearby' if district_championship_teams[team.team_key] < TRAVEL_DISTANCE else 'Travel')
        data['District Championship Distance'].append(district_championship_teams[team.team_key])
fum_team_data_df = pd.DataFrame(data)
del data
fum_team_data_df.to_csv('fum-team-data.csv')
